<a href="https://colab.research.google.com/github/TrinhThiToUyen/FinalProjectDataScience/blob/master/DataScience_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries

import pandas as pd
import json, codecs, os, csv
from time import time
import re
from lxml.etree import tostring
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
from keras.preprocessing import sequence
!pip install pyvi

Using TensorFlow backend.


     |████████████████████████████████| 8.8MB 3.4MB/s 
     |████████████████████████████████| 757kB 23.2MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df = pd.read_csv("/content/drive/My Drive/DataSicence/data_clean.csv",sep ='\t')

In [11]:
df['Full_Content'] = df['Title'] +'. '+ df['Content']
df_reduce = df.sample(5000)
df_reduce.Topic.value_counts()

Pháp luật    1134
Thế giới     1098
Thời sự      1013
Giáo dục      946
Khoa học      809
Name: Topic, dtype: int64

In [5]:
#@title 
from gensim.models import Word2Vec

input_gensim = []
contents = df['Full_Content'].tolist()
for content in contents:
    input_gensim.append(ViTokenizer.tokenize(content))
    
model = Word2Vec(input_gensim, size=128, window=5, min_count=0, workers=4, sg=1)
model.wv.save("/content/drive/My Drive/DataSicence/word2vec.model")

NameError: ignored

In [12]:
import gensim.models.keyedvectors as word2vec
from pyvi import ViTokenizer
model_embedding = word2vec.KeyedVectors.load('/content/drive/My Drive/DataSicence/word2vec.model')
word_labels = []
max_seq = 2560
embedding_size = 128

for word in model_embedding.vocab.keys():
    word_labels.append(word)
    
def comment_embedding(comment):
    matrix = np.zeros((max_seq, embedding_size))
    words = ViTokenizer.tokenize(comment)
    lencmt = len(words)

    for i in range(max_seq):
        indexword = i % lencmt
        if (max_seq - i < lencmt):
            break
        if(words[indexword] in word_labels):
            matrix[i] = model_embedding[words[indexword]]
    matrix = np.array(matrix)
    return matrix

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:

train_data = []
label_data = []
train_data_raw = df_reduce['Full_Content'].to_numpy()
label_data_raw = df_reduce['Topic'].to_numpy()

for x in train_data_raw:
    train_data.append(comment_embedding(x))
train_data = np.array(train_data)



In [0]:
for y in label_data_raw:
    label_ = np.zeros(5)
    if y == "Pháp luật":
      label_[0] = 1
    elif y == "Thế giới":
      label_[1] = 1
    elif y == "Thời sự":
      label_[2] = 1
    elif y == "Giáo dục":
      label_[3] = 1
    elif y == "Khoa học":
      label_[4] = 1
    label_data.append(label_)

In [0]:
sequence_length = 2560
embedding_size = 128
num_classes = 5
filter_sizes = 3
num_filters = 150
epochs = 50
batch_size = 30
learning_rate = 0.01
dropout_rate = 0.5
x_train = train_data.reshape(train_data.shape[0], sequence_length, embedding_size, 1).astype('float32')
y_train = np.array(label_data)


In [19]:
# Define model
model = keras.Sequential()

model.add(layers.Convolution2D(num_filters, (filter_sizes, embedding_size),
                        padding='valid',
                        input_shape=(sequence_length, embedding_size, 1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(18, 1)))
model.add(layers.Dropout(dropout_rate))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

# Train model
adam = tf.train.AdamOptimizer()
model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 2558, 1, 150)      57750     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 142, 1, 150)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 142, 1, 150)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 21300)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               2726528   
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645       
Total params: 2,784,923
Trainable params: 2,784,923
Non-trainable params: 0
____________________________________________

In [20]:
y_train = y_train.astype('float32')
y_train.dtype

dtype('float32')

In [21]:
model.fit(x = x_train[:4000], y = y_train[:4000], batch_size = batch_size, verbose=1, epochs=epochs, validation_data=(x_train[4000:5000], y_train[4000:5000]))

model.save('/content/drive/My Drive/DataSicence/models.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 4000 samples, validate on 1000 samples
Epoch 1/50
4000/4000 [==============================] - 17s 4ms/sample - loss: 0.4338 - acc: 0.8127 - val_loss: 0.3721 - val_acc: 0.8372
Epoch 2/50
4000/4000 [==============================] - 16s 4ms/sample - loss: 0.3196 - acc: 0.8630 - val_loss: 0.3270 - val_acc: 0.8612
Epoch 3/50
4000/4000 [==============================] - 16s 4ms/sample - loss: 0.2440 - acc: 0.9018 - val_loss: 0.3324 - val_acc: 0.8712
Epoch 4/50
4000/4000 [==============================] - 16s 4ms/sample - loss: 0.2071 - acc: 0.9169 - val_loss: 0.3247 - val_acc: 0.8676
Epoch 5/50
4000/4000 [==============================] - 16s 4ms/sample - loss: 0.1862 - acc: 0.9240 - val_loss: 0.3353 - val_acc: 0.8684
Epoch 6/50
4000/4000 [==============================] - 16s 4ms/sample - loss: 0.1733 - acc: 0.9304 - val_loss: 0.3356 - val_acc: 0.8718
Epoch 7/50
4000/4000 [==============

In [0]:
dicTopic = {0:"Pháp luật",
       1:"Thế giới",
       2: "Thời sự",
       3:"Giáo dục",
       4:"Khoa học"}
   

In [0]:
text1 = 'Bé sơ sinh bị rắn cắn tử vong HÀ TĨNH Bé Thái Hữu Tâm, 22 ngày tuổi, ở xã Thọ Điền, huyện Vũ Quang, đang nằm ngủ cạnh mẹ thì bị rắn cạp nia bò lên giường cắn tử vong, sáng 8/1. Lúc 5h sáng, chị Đinh Thị Huyền Trang (mẹ bé Tâm) nghe tiếng con trai khóc liền vùng dậy, phát hiện con rắn màu đen trắng đang bò trên giường nên hô hoán chồng vào dùng gậy đánh chết, đồng thời đưa con trai tới nhà thầy lang trong huyện để cứu chữa. Ông Nguyễn Đăng Nhàn, Chủ tịch UBND xã Thọ Điền, cho biết khi tới nhà thầy lang, người bé Tâm đã cứng, được cho uống thuốc song không qua khỏi. Cán bộ Trạm y tế xã kiểm tra vết thương, xác định bé trai tử vong do bị rắn cạp nia dài hơn một mét cắn trúng tay phải, Vợ chồng chị Trang có ba người con, bé Tâm là con trai út, mới sinh được 22 ngày. Gia đình mới xây nhà, phòng ngủ rất kín đáo, xung quanh khuôn viên có một vài cây bụi, song không quá rậm rạp. Chủ tịch UBND xã Thọ Điền cho hay, vài năm trước, xã cũng có một người đang nằm ngủ bị rắn độc bò vào giường cắn bị thương.'

In [24]:
#text = 'Lừa bán đất ven sông Hàn. ĐÀ NẴNGVũ Xuân Ngữ (30 tuổi) lừa 3 người với số tiền hơn 7,1 tỷ đồng bằng việc bán đất "ma" ven sông Hàn.Ngày 8/1, công an Đà Nẵng đã khởi tố, bắt Ngữ về hành vi lừa đảo chiếm đoạt tài sản. '+'Theo điều tra, Ngữ bịa chuyện có nhiều lô đất ven sông Hàn (đoạn thuộc đường Như Nguyệt, phường Thuận Phước, quận Hải Châu), nhờ người quen làm môi giới.'+'Tháng 3/2018, Ngữ thỏa thuận bán 4 lô đất cho chị Cúc (28 tuổi, trú TP HCM),  thoả thuận bán 4 lô đất với giá mỗi lô 800 triệu đồng cho diện tích 100 m2, rẻ hơn nhiều lần giá thị trường. Ngữ ba lần nhận tiền đặt cọc, tổng cộng 1,5 tỷ đồng.'+'Ngữ sau đó chở chị Cúc đến xem đất, hẹn ngày tách thửa sẽ hoàn tất thủ tục mua bán. Đến tháng 2/2019, Ngữ vẫn không làm được sổ đỏ nhưng tiếp tục yêu cầu chị Cúc chuyển thêm hơn 300 triệu đồng. Biết bị lừa, nạn nhân tố cáo đến công an.'+'Ngữ khai đã lừa bán đất cho hai người khác bằng thủ đoạn tương tự, chiếm đoạt 5,3 tỷ đồng. '
maxtrix_embedding = np.expand_dims(comment_embedding(text1), axis=0)
maxtrix_embedding = np.expand_dims(maxtrix_embedding, axis=3)

result = model.predict(maxtrix_embedding)
result = np.argmax(result)
print("Label predict: ", dicTopic[result])

Label predict:  Thời sự


In [25]:
y_train.shape

(5000, 5)

In [0]:
test_data = []
label_test_data = []
train_data_raw = df['Full_Content'][20000:22000].to_numpy()
label_data_raw = df['Topic'][20000:22000].to_numpy()

for x in train_data_raw:
    test_data.append(comment_embedding(x))
test_data = np.array(test_data)
for y in label_data_raw:
    label_ = np.zeros(5)
    if y == "Pháp luật":
      label_[0] = 1
    elif y == "Thế giới":
      label_[1] = 1
    elif y == "Thời sự":
      label_[2] = 1
    elif y == "Giáo dục":
      label_[3] = 1
    elif y == "Khoa học":
      label_[4] = 1
    label_test_data.append(label_)

In [33]:
x_test = test_data.reshape(test_data.shape[0], sequence_length, embedding_size, 1).astype('float32')
y_test = np.array(label_test_data)
score=model.evaluate(x_test, y_test, verbose=2)
print(score)

2000/2000 - 2s - loss: 0.5173 - acc: 0.8666
[0.5172941989898682, 0.8666]
